# Exponential kernel with fixed beta

In [1]:
from mlpp.pp import Hawkes, HawkesKernelExp
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
from mlpp.base.utils import TimeFunction
import numpy as np

from mlpp.optim.model import ModelHawkesFixedBeta
from mlpp.optim.solver import Fista
from mlpp.optim.prox import ProxZero, ProxPositive

from scipy.misc import derivative
from scipy.optimize import minimize

## MemeTracker

In [2]:
dir_name = 'datasets/memetracker/top10_1months_start2009-04'

In [3]:
filename = dir_name+'/process_with_cumul.pkl.gz'
import gzip, pickle
f = gzip.open(filename,'r')
cumul = pickle.load(f,encoding='latin1')
f.close()

In [4]:
f = gzip.open(dir_name+'/true_G.pkl.gz','r')
true_G = pickle.load(f)
f.close()
d = len(true_G)
true_G = np.array(true_G).reshape(d,d)

## Simple simulation

In [5]:
url = 'https://s3-eu-west-1.amazonaws.com/nphc-data/d10_2blocks_sym_with_Beta.pkl.gz'
from utils.loader import load_data
cumul, Beta = load_data(url)

In [ ]:
for i, process in enumerate(cumul.N):
    model = ModelHawkesFixedBeta(**{'beta':10}).fit(process)
    prox = ProxZero()
    fi = Fista(max_iter=2000, tol=1e-5, verbose=False).set_model(model).set_prox(prox)
    res = fi.solve(np.ones(d))
    mu[i] = res[0]
    alpha[i,:] = np.ones(d) * res[1]

## Hawkes creation

In [5]:
beta = 3

dim = 2
betas = beta*np.ones((dim, dim))
alphas = np.arange(1, dim*dim + 1).reshape((dim, dim)) / (dim*dim) / beta
mus = np.arange(1, dim + 1) / dim

hawkes = Hawkes(alphas=alphas, betas=betas, mus=mus)

print("mu vector")
print(mus)

print("\nkernels matrix")
print(hawkes.kernels)

print("\nspectral radius = %g" % hawkes.spectral_radius())

mu vector
[ 0.5  1. ]

kernels matrix
[[0.0833333*3*exp(-3*t) 0.166667*3*exp(-3*t)]
 [0.25*3*exp(-3*t) 0.333333*3*exp(-3*t)]]

spectral radius = 0.44769


## Hawkes simulation

In [6]:
run_time = 20000
%time hawkes.simulate(run_time)

ticks = hawkes.process

print("\nnumber of jumps :")
for i in range(dim):
    print(" - dimension %i : %i" % (i, len(ticks[i])))

tot_ticks = hawkes.n_total_jumps

CPU times: user 32.4 ms, sys: 1.53 ms, total: 33.9 ms
Wall time: 33.5 ms

number of jumps :
 - dimension 0 : 17533
 - dimension 1 : 36308


## Hawkes estimation

### with scipy

In [9]:
# In order to ensure positivity of parameters
model = ModelHawkesFixedBeta(beta, optimization_level=3).fit(ticks)
bnds = [(0.1, None) for _ in range(model.n_coeffs)]
%time result = minimize(lambda x: model.loss(x), 1*np.ones(model.n_coeffs), method='L-BFGS-B', bounds=bnds)

print(result)

CPU times: user 180 ms, sys: 2.84 ms, total: 183 ms
Wall time: 185 ms
     nit: 16
     jac: array([  1.45519152e-03,   1.49157131e-02,   5.19105379e+02,
         2.91038305e-03,  -1.81898940e-03,   1.35332812e-01])
       x: array([ 0.49879058,  1.00941354,  0.1       ,  0.16130576,  0.24326698,
        0.32655375])
 message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
 success: True
    nfev: 147
     fun: 29811.210533316775
  status: 0


In [10]:
coeffs = result.get('x')
print("original mu vector")
print(mus)
print("estimated mu vector")
mus_ = coeffs[0:dim]
print(mus_)

print("\noriginal alpha matrix")
print(alphas)
print("enstimated alpha matrix")
alphas_ = coeffs[dim:].reshape((dim, dim))
print(alphas_)

original mu vector
[ 0.5  1. ]
estimated mu vector
[ 0.49879058  1.00941354]

original alpha matrix
[[ 0.08333333  0.16666667]
 [ 0.25        0.33333333]]
enstimated alpha matrix
[[ 0.1         0.16130576]
 [ 0.24326698  0.32655375]]


### with custom Fista algorithm

In [11]:
model = ModelHawkesFixedBeta(beta, optimization_level=2).fit(ticks)
prox = ProxZero()

fista = Fista(max_iter=60, print_every=10, y_prox=ProxPositive()).set_model(model).set_prox(prox)
fista.params.do_linesearch = True

coeffs = fista.solve(1*np.ones(model.n_coeffs), step=1e4)

Launching the solver Fista...
  n_iter  |    obj    |    step   |  rel_obj 
        0 |  4.59e+04 |  1.86e-05 |  3.88e-01
       10 |  2.98e+04 |  9.31e-06 |  3.88e-04
       20 |  2.98e+04 |  9.31e-06 |  1.69e-07
       30 |  2.98e+04 |  4.66e-06 |  1.06e-08
       40 |  2.98e+04 |  9.31e-06 |  1.37e-10
       50 |  2.98e+04 |  9.31e-06 |  2.34e-12
       60 |  2.98e+04 |  9.31e-06 |  1.30e-13
Done solving using Fista in 0.33470797538757324 seconds


In [12]:
print("original mu vector")
print(mus)
print("estimated mu vector")
mus_ = coeffs[0:dim]
print(mus_)

print("\noriginal alpha matrix")
print(alphas)
print("enstimated alpha matrix")
alphas_ = coeffs[dim:].reshape((dim, dim))
print(alphas_)

original mu vector
[ 0.5  1. ]
estimated mu vector
[ 0.50748853  1.00941357]

original alpha matrix
[[ 0.08333333  0.16666667]
 [ 0.25        0.33333333]]
enstimated alpha matrix
[[ 0.0802487   0.16462255]
 [ 0.24326716  0.32655214]]


In [13]:
import numpy as np
from scipy.misc import derivative
from mlpp.pp.hawkes import Hawkes

dim = 2

# We ensure our parameters are in a fixed interval in order to have a simulation without too many points
alphas = np.random.random((dim, dim)) / 2
beta = 3 * np.random.random() + 2
betas = beta * np.ones((dim, dim))
mus = np.random.random(dim) / 3

hawkes = Hawkes(alphas=alphas, betas=betas, mus=mus)
run_time = 1000
hawkes.simulate(run_time)

ticks = hawkes.process


model = ModelHawkesFixedBeta(beta, optimization_level=1).fit(ticks)
model3 = ModelHawkesFixedBeta(beta, optimization_level=3).fit(ticks)
n_params = model.n_coeffs

# The vector at which we will compute the gradient
picked_coeffs = np.ones(n_params) + np.random.random(n_params)

grad_value = model3.grad(picked_coeffs)
print(grad_value)

# this will be used to create lambda function on the fly
def grad_for_coordinate(x: float, coordinate: int, coeffs: np.ndarray):
    c = coeffs.copy()
    c[coordinate] = x
    return model3.loss(c)

# because it is hard to know which range will be the right one, we test many of them
dx_range = [10.0 ** -i for i in range(0, 13)]
for i in range(n_params):
    f = lambda x: grad_for_coordinate(x, i, picked_coeffs)
    smallest_error = np.inf
    for dx in dx_range:
        smallest_error = min(smallest_error,
                           np.abs(derivative(f, picked_coeffs[i], dx=dx) - grad_value[i]))
    print(smallest_error)

print('----')
# this will be used to create lambda function on the fly
def grad_for_coordinate(x: float, coordinate: int, coeffs: np.ndarray):
    c = coeffs.copy()
    c[coordinate] = x
    return model.loss(c)

# because it is hard to know which range will be the right one, we test many of them
dx_range = [10.0 ** -i for i in range(0, 13)]
for i in range(n_params):
    f = lambda x: grad_for_coordinate(x, i, picked_coeffs)
    smallest_error = np.inf
    for dx in dx_range:
        smallest_error = min(smallest_error,
                           np.abs(derivative(f, picked_coeffs[i], dx=dx) - grad_value[i]))
    print(smallest_error)

print('----')    
grad_value = model.grad(picked_coeffs)
print(grad_value)

# because it is hard to know which range will be the right one, we test many of them
dx_range = [10.0 ** -i for i in range(0, 13)]
for i in range(n_params):
    f = lambda x: grad_for_coordinate(x, i, picked_coeffs)
    smallest_error = np.inf
    for dx in dx_range:
        smallest_error = min(smallest_error,
                           np.abs(derivative(f, picked_coeffs[i], dx=dx) - grad_value[i]))
    print(smallest_error)

[ 838.65613363  916.13124389  311.7767524   191.51333085  328.50975671
  180.02459445]
6.1546870711e-08
1.75548393599e-08
1.88641138266e-08
8.49033199302e-09
2.27672671826e-08
5.334254638e-08
----
6.1546870711e-08
1.75548393599e-08
1.88641138266e-08
8.49033199302e-09
2.27672671826e-08
5.334254638e-08
----
[ 838.65613363  916.13124389  311.7767524   191.51333085  328.50975671
  180.02459445]
6.1546870711e-08
1.75548393599e-08
1.88641138266e-08
8.49033199302e-09
2.27672671826e-08
5.334254638e-08


In [14]:
%timeit grad_value = model.grad(picked_coeffs)

10000 loops, best of 3: 25.3 µs per loop


In [15]:
%timeit grad_value = model3.grad(picked_coeffs)

10000 loops, best of 3: 25.1 µs per loop
